In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import norm

In [11]:
# List of tickers
tickers = ['rog', 'cfr', 'zurn']

# Date range for filtering
start_date = '2023-10-25'
end_date = '2024-10-25'

# Function to read and filter CSV files
def read_and_filter_csv(file_path, date_col='Date', price_col='Last Price'):
    df = pd.read_csv(file_path)
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    return df[(df[date_col] >= start_date) & (df[date_col] <= end_date)]

# Initialize the combined DataFrame
combined_df = pd.DataFrame()

# Process tickers for prices and implied volatility
for t in tickers:
    # Read and filter price data
    price_df = read_and_filter_csv(f'../data/{t}.csv')
    price_df = price_df[['Date', 'Last Price']].rename(columns={'Last Price': f'{t.upper()}_Last_Price'})
    
    # Read and filter implied volatility data
    ivol_df = read_and_filter_csv(f'../data/{t}_ivol.csv', price_col='IVOL')
    ivol_column_name = ivol_df.columns[1]
    ivol_df = ivol_df[['Date', ivol_column_name]].rename(columns={ivol_column_name: f'{t.upper()}_IVOL'})

    # Merge the dataframes
    combined_df = pd.merge(combined_df, price_df, on='Date', how='outer') if not combined_df.empty else price_df
    combined_df = pd.merge(combined_df, ivol_df, on='Date', how='outer')

# Read and filter risk-free rate data
risk_free_df = read_and_filter_csv('../data/risk_free.csv')
risk_free_column_name = risk_free_df.columns[1]
risk_free_df = risk_free_df[['Date', risk_free_column_name]].rename(columns={risk_free_column_name: 'Risk_Free_Rate'})

# Merge risk-free rate data
combined_df = pd.merge(combined_df, risk_free_df, on='Date', how='outer')

# Sort and reset index
combined_df = combined_df.sort_values('Date').reset_index(drop=True)

# Show the combined DataFrame
print(combined_df.head())

        Date  ROG_Last_Price  ROG_IVOL  CFR_Last_Price  CFR_IVOL  \
0 2023-10-25          237.55   17.6858          107.00   33.0793   
1 2023-10-26          237.25   17.9504          104.45   33.5406   
2 2023-10-27          237.75   19.6028          104.40   33.4355   
3 2023-10-30          241.30   19.3503          104.75   33.1531   
4 2023-10-31          233.85   19.2518          106.90   32.7788   

   ZURN_Last_Price  ZURN_IVOL  Risk_Free_Rate  
0            426.4    15.7900          1.4594  
1            426.8    15.9736          1.4508  
2            424.4    16.3945          1.4478  
3            428.2    15.9477          1.4623  
4            430.6    15.7948          1.4489  
